In [1]:
using Pkg
# Pkg.activate("/home/mat/.julia/environments/v1.5/Project.toml")
Pkg.activate("/media/mat/HDD/QROMP/")

 Activating environment at `/media/mat/HDD/QROMP/Project.toml`


In [2]:
using Revise
using LinearAlgebra
using Statistics
using BenchmarkTools
using QROMP
using BlockDiagonal

┌ Info: Precompiling QROMP [2ae2181e-4684-4b2c-a413-203f2dd831ec]
└ @ Base loading.jl:1278


In [106]:
ψ = randn(10, 5)
ϕ = randn(10)
u = randn(10)
c = ψ\u;
c0 = deepcopy(c)
F = qrfactUnblocked(ψ);
m, n = size(ψ)

(10, 5)

In [107]:
@show c

c = [-0.13968331152188715, 0.20744740760361802, -0.2736449288487641, -0.2452211750762385, 0.16003668781359986]


5-element Array{Float64,1}:
 -0.13968331152188715
  0.20744740760361802
 -0.2736449288487641
 -0.2452211750762385
  0.16003668781359986

In [108]:
@time idx, cOMP, cLS, FOMP, P, ek = qromp(ψ, c, u)

  0.000092 seconds (180 allocations: 409.625 KiB)


([2, 3, 4, 5, 1], [0.2074474076036181, -0.2736449288487641, -0.24522117507623847, 0.1600366878135997, -0.139683311521887], [-0.13968331152188715, 0.20744740760361802, -0.2736449288487641, -0.2452211750762385, 0.16003668781359986], QR{Float64,Array{Float64,2}}([3.714113039706446 -1.2696254436648913 … -1.3281613984654776 -1.5238595911560058; 0.444972551783379 4.418106356720047 … 0.49235485980423194 -0.4126510428589262; … ; -0.24018268993800543 -0.1204941598510186 … -0.17368559274615442 0.45337209708260606; 0.16561828814310414 -0.008516419007958582 … -0.09349298486369467 -0.529150340364789], [1.2057475927680779, 1.250222479213345, 1.2686599866148927, 1.4491185153710713, 1.0586462500891272]), [0.0 0.0 … 0.0 1.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0], [0.0, 0.0, 0.0, 0.0, 1.0])

In [111]:
cLS

5-element Array{Float64,1}:
 -0.13968331152188715
  0.20744740760361802
 -0.2736449288487641
 -0.2452211750762385
  0.16003668781359986

In [112]:
cOMP

5-element Array{Float64,1}:
  0.2074474076036181
 -0.2736449288487641
 -0.24522117507623847
  0.1600366878135997
 -0.139683311521887

In [110]:
FOMP.Q*FOMP.R*P'-ψ

10×5 Array{Float64,2}:
 -1.38778e-17   2.22045e-16   4.44089e-16  -1.11022e-16  -4.44089e-16
  0.0           0.0          -2.22045e-16   0.0          -2.22045e-16
 -2.22045e-16   0.0           2.22045e-16   2.77556e-17   0.0
 -4.44089e-16   0.0          -2.22045e-16   0.0           0.0
  0.0           0.0           0.0           0.0          -2.22045e-16
  1.11022e-16   0.0           0.0           5.55112e-17  -1.11022e-16
 -2.22045e-16  -1.11022e-16   1.11022e-16   0.0           0.0
  0.0           0.0           0.0           5.55112e-17   5.55112e-17
  0.0           0.0           0.0           0.0           5.55112e-17
  0.0           0.0           5.55112e-17   0.0          -1.11022e-16

In [74]:
## Orthogonal Matching Pursuit
idx_OMP = Int64[]
ϵ = 1e-1
res_OMP = deepcopy(u)
Γ_OMP = Float64[]
dict_OMP = collect(1:n);
entry = 0.0;
counter = 0
new_idx = 1

normcol = norm.(eachcol(ψ))
@show normcol
c_OMP = zero(ψ\u)

@inbounds while counter < size(ψ,2) #&& norm(res_OMP)>ϵ
    counter += 1
    # Compute 
    entry = 0.0

    for (i, idx) in enumerate(dict_OMP)
        col = view(ψ,:,idx)
        ratio = (dot(col,res_OMP)/normcol[idx])^2
        # Find the largest entry and its index on the fly
        if ratio > entry
            entry = ratio
            new_idx = idx
        end
    end
    @show entry, new_idx
    filter!(x-> x != new_idx, dict_OMP)
    push!(idx_OMP, copy(new_idx))
#     @assert sort(union(idx_OMP, dict_OMP)) == collect(1:nblob)
    ψ_OMP = view(ψ,:,sort(idx_OMP))
    c_OMP = ψ_OMP\u
    res_OMP = u-ψ_OMP*c_OMP
    @show norm(res_OMP)/norm(u)
end

normcol = [2.295229894632921, 3.8577630786296475, 2.1570039960723766, 5.01183467379606, 2.944282239584703]
(entry, new_idx) = (5.290084514515703, 1)
norm(res_OMP) / norm(u) = 0.8074598006228048
(entry, new_idx) = (3.110374459725085, 3)
norm(res_OMP) / norm(u) = 0.6656023726192064
(entry, new_idx) = (1.6242063603795405, 4)
norm(res_OMP) / norm(u) = 0.5759683159681794
(entry, new_idx) = (1.0246926350150303, 2)
norm(res_OMP) / norm(u) = 0.4906497491286512
(entry, new_idx) = (0.044705391291883306, 5)
norm(res_OMP) / norm(u) = 0.48499593629880516


In [75]:
c_OMP

5-element Array{Float64,1}:
 -0.9870716766793397
  0.385696490747567
 -0.3994939702253127
 -0.3342135128920916
  0.13469352291508357

In [76]:
cLS

5-element Array{Float64,1}:
 -0.9870716766793397
  0.385696490747567
 -0.3994939702253127
 -0.3342135128920916
  0.13469352291508357

In [35]:
Matrix(F.Q)*Matrix(F.Q)'

100×100 Array{Float64,2}:
  0.109049      0.0121885    -0.0247129   …   0.0528155    0.0340403
  0.0121885     0.0834409     0.0120749       0.0337248    0.0107088
 -0.0247129     0.0120749     0.11785         0.00792615  -0.0205
 -0.0265195    -0.0176442     0.0559337      -0.00618252  -0.104716
  1.3746e-5     0.0421841     0.0546239       0.0373985    0.0932801
  0.0336775    -0.0062833    -0.079351    …  -0.0247392    0.0279264
  0.0555426    -0.00431462   -0.0141412       0.0145937    0.0471913
  0.0400674     0.00612087   -0.0226838       0.0203272   -0.0702643
  0.0182465    -0.0171899     0.0323537      -0.0153783    0.0141202
 -0.00458054    0.0328571     0.0507023      -0.0111118   -0.0374159
  0.00219689   -0.00106086    0.0097368   …   0.0209563    0.0111246
  0.0445891    -0.0022222     0.0288602       0.00490472   0.0383929
  0.0224388    -0.00176537   -0.0535356      -0.00801796   0.0413919
  ⋮                                       ⋱               
 -0.0214373     0.0052

In [30]:
ψ*c-u;

In [15]:
?normsqu

search: UniformScaling set_zero_subnormals get_zero_subnormals

Couldn't find normsqu
Perhaps you meant norm or normpath


No documentation found.

Binding `normsqu` does not exist.


In [29]:
(Matrix(F.Q)[:,1:9]*Matrix(F.Q)[:,1:9]' + Matrix(F.Q)[:,10]*Matrix(F.Q)[:,10]' - I)*u

100-element Array{Float64,1}:
  0.3758311589958424
  1.0585258076048505
  0.4447818407388319
  0.008553486267878399
  0.5800017631473279
  3.188412904859621
 -0.429577451836499
  0.208439956957505
 -0.08090965585245928
 -0.6038900950214523
  1.329789864255415
 -0.4949821463908698
 -0.7068285764729044
  ⋮
  0.044066172301584106
  1.0246146496589115
  0.1878612517179839
 -0.4173313397315703
  0.8272517442787691
  2.206261988756783
  0.34520276399716154
 -0.23646085941167977
  0.7055617064816128
  0.7709500107352945
 -2.4601768968417264
  0.34420330677473154

In [25]:
Matrix(F.Q)'*u

10-element Array{Float64,1}:
 -3.090247135269122
 -1.6048265352544309
  0.03897833173484622
 -0.5814289181053852
 -0.7094619236376158
 -1.8678473043938453
  0.10922273046087827
  0.026014165881648843
  1.1469596769660972
 -0.20439148335509472

In [19]:
(Matrix(F.Q)*Matrix(F.Q)'-I)*u

100-element Array{Float64,1}:
  0.3758311589958424
  1.0585258076048505
  0.4447818407388319
  0.008553486267878371
  0.5800017631473279
  3.188412904859621
 -0.429577451836499
  0.208439956957505
 -0.08090965585245928
 -0.6038900950214522
  1.329789864255415
 -0.4949821463908697
 -0.7068285764729044
  ⋮
  0.044066172301584106
  1.0246146496589115
  0.1878612517179839
 -0.4173313397315703
  0.8272517442787691
  2.2062619887567827
  0.3452027639971615
 -0.23646085941167988
  0.7055617064816128
  0.7709500107352945
 -2.4601768968417264
  0.34420330677473154

In [18]:
Matrix(F.Q)'*Matrix(F.Q)

10×10 Array{Float64,2}:
  1.0          -2.97047e-18   1.36488e-17  …   3.5366e-17   -7.8012e-18
 -2.97047e-18   1.0          -6.15815e-18     -6.53399e-17   3.4025e-17
  1.36488e-17  -6.15815e-18   1.0             -2.49018e-17  -4.29891e-17
 -1.30726e-17  -7.77362e-17  -4.15673e-17      2.30869e-17  -7.65614e-17
  4.07212e-17   2.5782e-17   -4.42018e-17      9.67024e-18  -3.81687e-17
 -4.78339e-17  -7.8102e-18   -1.00966e-17  …  -2.08837e-17   1.46289e-18
 -4.15594e-17   1.12025e-18  -5.314e-17        3.33712e-17  -4.02865e-17
  3.59917e-17   1.26118e-17   1.0058e-16       2.78907e-17  -9.61818e-18
  3.5366e-17   -6.53399e-17  -2.49018e-17      1.0          -2.36955e-17
 -7.8012e-18    3.4025e-17   -4.29891e-17     -2.36955e-17   1.0

In [8]:
@time Matrix(F.Q)

  0.000025 seconds (6 allocations: 51.094 KiB)


100×10 Array{Float64,2}:
 -0.0601161    0.0131009   -0.204613    …   0.000803645  -0.146612
 -0.139088    -0.0112863    0.0315648       0.0758086    -0.104971
 -0.127987    -0.0442696   -0.136143       -0.130357      0.00237226
  0.0712545    0.0542568   -0.142453        0.0260971    -0.139892
 -0.0799792   -0.0234555    0.0847968       0.0048141     0.244689
 -0.115188    -0.0597474    0.101312    …  -0.130346      0.104948
  0.286876    -0.0588683   -0.0310429      -0.0740838     0.0468238
  0.0743137    0.048656    -0.0852818       0.0803014    -0.0448807
 -0.0758305   -0.043444    -0.248028       -0.0488789     0.120366
  0.0470619   -0.0754272    0.0280316      -0.0710276     0.00604375
 -0.100689    -0.0678126   -0.0281702   …   0.11979       0.0909163
 -0.0757729   -0.00886938   0.0573663      -0.0120697    -0.0452479
 -0.00954337   0.120647     0.0617183       0.0216977     0.0604418
  ⋮                                     ⋱                
 -0.0169328   -0.0404478   -0.132958 

In [15]:
# @show norm(F.Q*F.R-ψ)
# @show norm(F.Q'*F.Q-I)
# @show norm(F.Q*F.Q'-I)

In [122]:
@btime begin
    A = randn(500,1)
    ϕ = randn(500)
    for i=1:99
    A = hcat(A, ϕ)
    end
end

  1.110 ms (197 allocations: 19.28 MiB)


In [109]:
@btime begin
    G = qrfactUnblocked(view(ψ,:,1:1))
    for i=2:20
    G = updateqrfactUnblocked!(G, view(ψ,:,i))
    end
end

  39.147 μs (181 allocations: 812.48 KiB)


In [117]:
# https://discourse.julialang.org/t/extract-submatrix-from-qr-factor-is-slow-julia-1-4/36582/4
# We want to avoid to form the Q matrix, instead, we want to leverage the Householder reflections
# For steps 6. and 11. of Baptista et al. 2019
@time q = (F.Q*ek) #2.852 μs (4 allocations: 1.17 KiB)
@time (q'*u)*q #102.923 ns (3 allocations: 928 bytes)
@time ((F.Q[:,end])'*u)*F.Q[:,end] #559.818 μs (610 allocations: 399.67 KiB);

  0.000030 seconds (4 allocations: 1.172 KiB)
  0.000008 seconds (3 allocations: 928 bytes)
  0.000656 seconds (611 allocations: 399.688 KiB)
